In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import glob


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df_info  = pd.read_csv('SPindex.csv')
df_info = df_info.set_index('Symbol')
df_info.head()

,Security,GICS Sector,GICS Sub Industry,CIK
Symbol,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,66740
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1800
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,1467373


In [3]:
def check_file_existance(file_ids , tmpl):
    files = []
    path = tmpl.format(subject = '*')
    directory_files = glob.glob(path , recursive=True)

    for f_name in file_ids:
        filename = tmpl.format(subject = f_name)
        if filename in directory_files:
            files.append((f_name , filename))
        else:
            files.append((f_name , 'None'))
    return files


## Load Data From Files

In [4]:
def loadData(files , from_date , to_date):
    total  =  []
    
    for i in range(len(files)):
        file_id = files[i][0]
        filename = files[i][1]


        df = pd.read_csv(filename)
        df = df.loc[ (df['Date'] >= from_date) & (df['Date'] <= to_date)]
        
        df.insert(0,'Symbol',file_id)
        total.append(df)
    
    return total

In [5]:
symbol_names = list(df_info.index)
files = check_file_existance(symbol_names , 'stocks\{subject}.csv')
total = loadData(files , '2019-01-01' , '2019-12-31')

## Create File Base Moving Average of price - Index By Date

In [29]:
def create_price_df_baseDate(total):
    for df in total:
        df.dropna(inplace = True)       
        df['Close_Rolling'] = df['Adj Close'].rolling(3).mean()
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Date', columns='Symbol', values='Close_Rolling')
    
    
    # column name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset.columns[dataset.isna().any()].tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.columns:
            dataset.drop(symbol , axis = 1, inplace = True)       
        
    return dataset

In [30]:
data_perdate = create_price_df_baseDate(total)
data_perdate.to_csv('stocks/total_data_price_Date.csv')
data_perdate.head()

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2019-02-05,75.290733,36.057185,159.919983,167.273066,73.041997,82.278748,341.923330,71.199201,152.973109,252.313339,...,108.702604,69.626170,42.276562,27.523523,69.306986,92.441663,116.516917,176.826665,46.504528,86.513842
2019-02-06,75.376447,36.303276,159.315948,169.795481,72.607511,83.211746,339.823334,71.568311,153.763087,254.636673,...,109.557892,69.423136,42.666055,27.814152,69.769666,92.591799,117.038434,178.726664,46.724112,86.659424
2019-02-07,75.145678,36.135525,159.906713,169.694194,72.803335,83.649091,343.556661,71.646706,153.315857,254.303340,...,110.502780,69.380068,42.689161,27.909963,69.786072,92.578743,117.107752,178.806661,46.816405,86.407969
2019-02-08,74.977549,35.829624,160.222005,168.701197,72.797213,83.176109,342.533325,71.718567,152.751114,255.030004,...,111.388814,69.137454,42.497714,27.945094,69.736849,92.595062,117.853732,177.983327,46.883235,86.229299
2019-02-11,74.905022,35.773708,162.847234,167.367009,73.008336,82.204234,345.913330,71.734901,152.150472,256.376673,...,111.594795,69.033676,42.398692,27.900382,69.687632,92.160962,119.190544,177.189997,47.023261,85.795868


## Create File Base Moving Average of price - Index By Symbol

In [31]:
def create_price_df_baseSymbol(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close_Rolling'] = df['Adj Close'].rolling(3).mean()        
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Symbol', columns='Date', values='Close_Rolling')
    
    
    #symbol name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset[dataset.isna().any(axis=1)].index.tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.index:
            dataset.drop(symbol , axis = 0, inplace = True)
        
        
    return dataset

In [32]:
data_perSymbol = create_price_df_baseSymbol(total)
data_perSymbol.to_csv('stocks/total_data_price_Symbol.csv')
data_perSymbol.head()

Date,2019-02-07,2019-02-08,2019-02-11,2019-02-12,2019-02-13,2019-02-14,2019-02-15,2019-02-19,2019-02-20,2019-02-21,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
Symbol,,,,,,,,,,,,,,,,,,,,,
A,75.145678,74.977549,74.905022,75.478650,75.956675,76.266566,76.672066,76.761075,77.268776,77.133611,...,83.646706,83.553805,83.573713,83.985128,84.592298,84.854411,84.940674,84.993759,84.920527,84.933851
AAL,36.135525,35.829624,35.773708,35.635560,35.704636,35.546753,35.316507,35.073104,34.727735,34.704711,...,27.790633,27.963329,28.172555,28.424956,28.856693,29.065921,29.275147,28.989536,28.697282,28.368497
AAP,159.906713,160.222005,162.847234,164.427007,165.771154,166.969269,167.666239,167.891917,166.982544,164.101756,...,155.066177,156.003169,157.013077,157.079549,157.744339,159.103836,159.486094,158.947617,158.465642,158.761475
AAPL,169.694194,168.701197,167.367009,167.590144,167.514664,167.964223,167.810003,168.056107,168.459712,168.669718,...,277.063985,278.586243,278.639313,278.317607,279.730418,281.139913,284.612254,286.535797,288.940236,290.180583
ABBV,72.803335,72.797213,73.008336,73.240878,73.583570,73.813049,74.094543,74.011930,73.849765,73.207219,...,86.636609,87.119456,86.905579,86.649572,86.947703,87.297684,87.472677,87.132416,86.701421,86.283389


## Create File Base on change of price - Index By Symbol

In [16]:
def create_change_df_baseSymbol(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close_Rolling'] = df['Adj Close'].rolling(3).mean()
        df['Close1']  = df['Close_Rolling'].shift()
        df['Change'] = np.round(df['Close_Rolling'] - df["Close1"] , 2)
        #df['Close1']  = df['Adj Close'].shift()
        #df['Change'] = np.round(df['Adj Close'] - df["Close1"] , 2)   
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Symbol', columns='Date', values='Change')
    
    
    #symbol name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset[dataset.isna().any(axis=1)].index.tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.index:
            dataset.drop(symbol , axis = 0, inplace = True)        
        
    return dataset

In [17]:
data_perSymbol = create_change_df_baseSymbol(total)
data_perSymbol.to_csv('stocks/total_data_price_Change_Symbol.csv')
data_perSymbol.head()

Date,2019-01-31,2019-02-01,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-11,2019-02-12,2019-02-13,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
Symbol,,,,,,,,,,,,,,,,,,,,,
A,0.92,0.82,0.19,0.09,0.09,-0.23,-0.17,-0.07,0.57,0.48,...,-0.29,-0.09,0.02,0.41,0.61,0.26,0.09,0.05,-0.07,0.01
AAL,-0.26,-0.06,0.14,0.41,0.25,-0.17,-0.31,-0.06,-0.14,0.07,...,0.19,0.17,0.21,0.25,0.43,0.21,0.21,-0.29,-0.29,-0.33
AAP,1.44,1.21,0.77,0.75,-0.60,0.59,0.32,2.63,1.58,1.34,...,0.62,0.94,1.01,0.07,0.66,1.36,0.38,-0.54,-0.48,0.30
AAPL,3.31,3.87,1.96,2.53,2.52,-0.10,-0.99,-1.33,0.22,-0.08,...,2.97,1.52,0.05,-0.32,1.41,1.41,3.47,1.92,2.40,1.24
ABBV,0.96,0.93,-0.16,-0.18,-0.43,0.20,-0.01,0.21,0.23,0.34,...,0.44,0.48,-0.21,-0.26,0.30,0.35,0.17,-0.34,-0.43,-0.42


## Create file base on return - Index by symbol

In [27]:
def create_return_df_baseSymbol(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close1']  = df['Adj Close'].shift()
        df['Return'] = np.log(df['Adj Close']) - np.log(df["Close1"])
        df.dropna(inplace = True)
        
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Symbol', columns='Date', values='Return')
    
    
    #symbol name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset[dataset.isna().any(axis=1)].index.tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.index:
            dataset.drop(symbol , axis = 0, inplace = True)        
        
    return dataset

In [28]:
data_perSymbol = create_return_df_baseSymbol(total)
data_perSymbol.to_csv('stocks/total_data_return_Symbol.csv')
data_perSymbol.head()

Date,2019-02-01,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-11,2019-02-12,2019-02-13,2019-02-14,...,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30,2019-12-31
Symbol,,,,,,,,,,,,,,,,,,,,,
A,-0.000132,-0.000132,0.003676,-0.000131,-0.012794,0.006220,0.003687,0.012929,0.002333,-0.002982,...,-0.005938,-0.006213,0.012862,0.008014,0.000821,0.000469,0.001757,-0.000351,-0.003997,0.004818
AAL,0.009460,0.018384,0.006793,-0.004614,-0.016180,-0.004711,0.016256,-0.023225,0.012785,-0.002765,...,0.012812,-0.009950,0.019452,0.017017,0.008913,-0.004104,0.016653,-0.042340,-0.004935,0.013338
AAP,-0.001006,0.014420,0.000682,-0.026611,0.036901,-0.004423,0.016609,0.016576,-0.008554,0.013604,...,0.028768,0.000317,-0.009477,0.010426,0.011632,0.003744,-0.008129,-0.005793,0.004851,0.006515
AAPL,0.000481,0.028009,0.016965,0.000344,-0.019121,0.001174,-0.005768,0.008580,-0.004163,0.003637,...,0.001964,-0.002392,0.001000,-0.002073,0.016187,0.000950,0.019646,-0.000380,0.005918,0.007280
ABBV,0.002612,-0.024777,0.014663,-0.007684,0.001138,0.006296,0.001254,0.002004,0.010702,-0.003348,...,0.007242,-0.008361,-0.006289,0.005841,0.010694,-0.004442,-0.000223,-0.007038,-0.007652,0.000226
